# Exploring many different models and short-listing the best ones

We are going to proceed to use our training data and our training labels without the variables `city`,`Latitude`,`Longitude`,`change_hunits` `change_hunits`,`studio_1000_1499`, `studio_1500_more`,
       `studio_750_999`, `onebed_1000_1499`, `onebed_1500_more`,
       `onebed_750_999`, `twobed_1000_1499`, `twobed_1500_more`,
       `twobed_750_999`, `threebed_1000_1499`, `threebed_1500_more`,
       `threebed_750_999`

Next its recommended to try out some models from various categories of Machine Learning algorithms **Note best practice recommends that we cannot simply standardize all of the data once at the beginning and run cross validation on the standardized data. To do so would be allowing the model to peek at the validation set during training.** And so when measuring the performance its is best to standardize our data within each fold in the N-fold cross-validation being done.

In [1]:
import numpy as np
import pandas as pd
import os
np.random.seed(22)
#load data
def load_housing_data(housing_path):
    csv_path = os.path.join(housing_path,'datasets','california_housing.csv')
    return pd.read_csv(csv_path,index_col=0)  
housing = load_housing_data('C:\\Users\\Crist\\Towncharts\\California_Housing_Project\\')
housing = housing.rename(columns = {'percent_of_rent_to_total':'rent_home_percent'})


#make train and test set
housing = housing[pd.notnull(housing['med_rental_rate'])]
housing = housing.reset_index(drop=True)

housing['property_cost_cat'] = pd.cut(housing['med_homeval'],
                                  bins = [0,237150,389600,620350,np.inf],
                                  labels = [1,2,3,4])

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=22)
for train_index,test_index in split.split(housing,housing['property_cost_cat']):
    strat_train_set  = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

    
#Drop the property value category attribute now that its been used to create the test set and no longer is needed
for set_ in (strat_train_set, strat_test_set):
    set_.drop("property_cost_cat", axis=1, inplace=True)

In [2]:
from src.modeling import *

Here we train a list of different models in order to shortlist the top promising models.

In [3]:

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

lin_reg = LinearRegression()
print('Ordinary Least Squares Linear Regression 5 fold Cross-Validation Results: ')
print(lin_reg)
model_pipeline(lin_reg,strat_train_set,5)
print("------------------------------------------------------------------------------------------------------------------")
#Ridge Regression
ridge_reg = Ridge(alpha =1 , solver = 'cholesky')
print(ridge_reg)
print('Ridge Regression 5 fold Cross-Validation Results: ')
model_pipeline(ridge_reg,strat_train_set,5)
print("------------------------------------------------------------------------------------------------------------------")
#Lasso Regression
lasso_reg = linear_model.Lasso(alpha=0.1,tol = 0.09,random_state= 22)
print('Lasso Regression 5 fold Cross-Validation Results: ')
print(lasso_reg)
model_pipeline(lasso_reg,strat_train_set,5)
print("------------------------------------------------------------------------------------------------------------------")
#Elastic Net
elastic_reg = ElasticNet(alpha = 0.1,tol=0.01, l1_ratio = 0.5,random_state= 22)
print('Elastic Net 5 fold Cross-Validation Results: ')
print(elastic_reg)
model_pipeline(elastic_reg,strat_train_set,5)
print("------------------------------------------------------------------------------------------------------------------")
#Random Forest Regressor
forest_reg = RandomForestRegressor(n_estimators=100,random_state= 22)
print('RF-Regressor 5 fold Cross-Validation Results: ')
print(forest_reg)
model_pipeline(forest_reg,strat_train_set,5)

Ordinary Least Squares Linear Regression 5 fold Cross-Validation Results: 
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)
Scores [[228.70962322 176.52633545 224.31023843 193.71690671 239.92708296]]
Mean 212.6380373537479
Standard Deviation 23.66977971984869
------------------------------------------------------------------------------------------------------------------
Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='cholesky', tol=0.001)
Ridge Regression 5 fold Cross-Validation Results: 
Scores [[228.41757545 177.22545992 220.96739757 192.37498221 235.90847068]]
Mean 210.97877716511599
Standard Deviation 22.408754778383287
------------------------------------------------------------------------------------------------------------------
Lasso Regression 5 fold Cross-Validation Results: 
Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, 

<br>

Elastic Net is our winner at this point in time with  a mean RMSE of ~207 and std ~17 at this point the model has not achieved our goal of providing an estimate of the median rental rate that is less than 192 dollars. Lets us see if we can improve upon this.

A helpful thing to do here is to check the noise of the data and how much variance in median rental price our best model can explain currently

In [4]:
#explained variance
print('Explained variance with an Elastic Net regresion model')
print(elastic_reg)
model_pipeline(elastic_reg,strat_train_set,5,checknoise=True)

Explained variance with an Elastic Net regresion model
ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.5,
           max_iter=1000, normalize=False, positive=False, precompute=False,
           random_state=22, selection='cyclic', tol=0.01, warm_start=False)
Scores [[0.83867501 0.88496919 0.8248445  0.78324435 0.74007055]]
Mean 0.8143607210858583
Standard Deviation 0.04934350514441448


ElasticNet accounts for ~ 81% of the variation in our given training data set with a sd of 4%

Lets investigate the relative importance of each attribute 
for making accurate prediction of median rental rate for feature selection purposes to see if reducing the noise i.e. dropping uninformative attributes improves our predictions. We will use the Random Forest Feature Importances

In [5]:
#from src.modeling import prepdf_or_featureselection

In [6]:
my_X_train = (prepdf_or_featureselection(strat_train_set,prep=True))['mydata']
rf_reg = RandomForestRegressor(n_estimators=100,random_state=22)
rf_reg.fit(my_X_train,strat_train_set['med_rental_rate'].copy())
feature_importances = rf_reg.feature_importances_

#sorted(zip(feature_importances,(prepdf_or_fullpipeline(strat_train_set,onlyprep = True))['mydatacolumns']),reverse=True)

In [7]:
attributes = (prepdf_or_featureselection(strat_train_set,prep = True))['mydatacolumns']
importance_results = sorted(zip(feature_importances, attributes), reverse=True)

In [8]:
for i in importance_results:
    print(i)
print('Total attributes: ',len(importance_results))

(0.4887781138536719, 'med_owner_cost')
(0.21675649535699323, 'med_hcost_ownmortg')
(0.050201590357428647, 'med_real_estate_taxes')
(0.03587588836602332, 'owned_homes')
(0.0347595014957407, 'median_num_ofrooms')
(0.0324637039464469, 'med_homeval')
(0.025615844525265767, 'rent_home_percent')
(0.013820970048462571, 'housing_density')
(0.013256862434699219, 'med_hcost_own_wo_mortg')
(0.010148870136003196, 'household_size_for_renters')
(0.009420439127731117, 'med_year_renter_moved_in')
(0.008661245901148296, 'median_year_house_built')
(0.00801330904523532, 'med_hval_aspercentof_medearn')
(0.007343128332141725, 'area_total_sq_mi')
(0.007072207368963794, 'hcost_aspercentof_hincome_ownmortg')
(0.007069719982840823, 'med_year_moved_in_for_owners')
(0.005911235719694841, 'population')
(0.004938470868514768, 'housing_units')
(0.004910073403746763, 'household_size_of_howners')
(0.004566983165365608, 'med_own_cost_aspercentof_income')
(0.00409571060983884, 'family_members_per_hunit')
(0.00303642340

In [9]:
importance_df= pd.DataFrame(importance_results,columns = ['relative_importance','value'])
plt.figure(figsize = (10,5))
chart = sns.barplot(x = 'value',y = 'relative_importance',data = importance_df)
plt.xlabel('Attribute')
plt.ylabel('Relative Importance')
plt.xticks(rotation = 45,horizontalalignment = 'right')
plt.show()

NameError: name 'plt' is not defined

Median Owner cost and median housing cost for those who own a mortgage are our 2 most informative attributes in predicting the median rental rate.

What if you wanted to look at how attributes for a city contribute to our machine learning estimators model predictions for a given city. In this case the SHAP library along with our random forest model can be used to explain the estimators predictions for San Diego and Los Angeles

In [ ]:
city_df = (strat_train_set.copy()).reset_index(drop=True)
imputer = IterativeImputer(max_iter = 10 ,random_state =22,min_value=0)
housing_shap = strat_train_set.drop(['city','Latitude','Longitude','change_hunits','studio_1000_1499', 'studio_1500_more',
       'studio_750_999', 'onebed_1000_1499', 'onebed_1500_more',
       'onebed_750_999', 'twobed_1000_1499', 'twobed_1500_more',
       'twobed_750_999', 'threebed_1000_1499', 'threebed_1500_more',
       'threebed_750_999','med_rental_rate'],axis=1)

imputed_shap = imputer.fit_transform(housing_shap)
housing_shap = pd.DataFrame(imputed_shap,columns = housing_shap.columns)
housing_labels_shap = np.array(strat_train_set['med_rental_rate'].copy())

In [ ]:
san_diego = city_df.loc[city_df.city=='SanDiego'].index[0]
los_angeles = city_df.loc[city_df.city=='LosAngeles'].index[0]

In [ ]:
shap_rf = RandomForestRegressor(n_estimators=100,random_state= 22)
shap_rf.fit(housing_shap,housing_labels_shap)

In [ ]:
import shap
shap.initjs()
explainer = shap.TreeExplainer(shap_rf)
shap_values = explainer.shap_values(housing_shap)

shap.force_plot(explainer.expected_value,shap_values[san_diego,:],housing_shap.iloc[san_diego,:])

For San Diego the above explanation shows features each contributing to push the model output from the base value (the average model output over the training dataset we passed) to the model output. Features pushing the prediction higher are shown in red, those pushing the prediction lower are in blue

The actual value for San Diego is 

In [ ]:
city_df.loc[city_df.city == 'SanDiego','med_rental_rate'].values[0]

Our model is underpredicting

Now lets do the same for Los Angeles

In [ ]:
shap.initjs()
shap.force_plot(explainer.expected_value,shap_values[los_angeles,:],housing_shap.iloc[los_angeles,:])

The actual value for Los Angeles is 

In [ ]:
city_df.loc[city_df.city == 'LosAngeles','med_rental_rate'].values[0]

Coloring by Median Home Evaluation highlights that med_own_cost_aspercentof_income (median owner cost as percent of household income) had less impact on median rental rate for areas with a low median home evaluation even as it increased.

In [ ]:
shap.dependence_plot('med_own_cost_aspercentof_income',shap_values,housing_shap)

Using the information given to us by the random forest feature importance let us now reuse our `preparedf_or_featureselection` function to explore how many features GridSearchCV recommends.

In [10]:
feature_selection_results = (prepdf_or_featureselection(strat_train_set,myfeature_importances=feature_importances,prep=False))['myfeature_selection']

In [11]:
feature_selection_results['feature_selection__k']

24

Which are:

In [12]:
informative_features = [i[1] for i in sorted(zip(feature_importances, attributes), reverse=True)][0:feature_selection_results['feature_selection__k']]
informative_features

['med_owner_cost',
 'med_hcost_ownmortg',
 'med_real_estate_taxes',
 'owned_homes',
 'median_num_ofrooms',
 'med_homeval',
 'rent_home_percent',
 'housing_density',
 'med_hcost_own_wo_mortg',
 'household_size_for_renters',
 'med_year_renter_moved_in',
 'median_year_house_built',
 'med_hval_aspercentof_medearn',
 'area_total_sq_mi',
 'hcost_aspercentof_hincome_ownmortg',
 'med_year_moved_in_for_owners',
 'population',
 'housing_units',
 'household_size_of_howners',
 'med_own_cost_aspercentof_income',
 'family_members_per_hunit',
 'hcost_as_perc_of_hincome_womortg',
 'cluster2',
 'cluster1']

Apparently almost all features are useful except one (cluster 0). Lets recheck the models we tried out above now with our 24 features and see by having less noise i.e. reducing the number of irrelevant features the models is trying to fit we can reduce our MSE and see if there is a new winner

In [ ]:

lin_reg = LinearRegression()
print('Ordinary Least Squares Linear Regression 5 fold Cross-Validation Results: ')
print(lin_reg)
model_pipeline(lin_reg,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)
print("------------------------------------------------------------------------------------------------------------------")
#Ridge Regression
ridge_reg = Ridge(alpha =1 , solver = 'cholesky')
print(ridge_reg)
print('Ridge Regression 5 fold Cross-Validation Results: ')
model_pipeline(ridge_reg,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)
print("------------------------------------------------------------------------------------------------------------------")
#Lasso Regression
lasso_reg = linear_model.Lasso(alpha=0.1,tol = 0.09,random_state= 22)
print('Lasso Regression 5 fold Cross-Validation Results: ')
print(lasso_reg)
model_pipeline(lasso_reg,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)
print("------------------------------------------------------------------------------------------------------------------")
#Elastic Net
elastic_reg = ElasticNet(alpha = 0.1,tol = 0.01, l1_ratio = 0.5,random_state= 22)
print('Elastic Net 5 fold Cross-Validation Results: ')
print(elastic_reg)
model_pipeline(elastic_reg,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)
print("------------------------------------------------------------------------------------------------------------------")
#Random Forest Regressor
forest_reg = RandomForestRegressor(n_estimators=100,random_state= 22)
print('RF-Regressor 5 fold Cross-Validation Results: ')
print(forest_reg)
model_pipeline(forest_reg,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)

As expected reducing the number of features down by just one does not improve our RMSE 

<a id="tuning"></a>

# Fine-tuning models and combine them into a great solution

### Tuning the hyper-parameters of the best estimators

Elastic Net Grid Search

To understand the effects of the parameters of ElasticNet this [link](https://stats.stackexchange.com/questions/84012/choosing-optimal-alpha-in-elastic-net-logistic-regression) was very helpful!

In [ ]:
%%capture
prep_pipeline = Pipeline([
    ('imputer',IterativeImputer(max_iter = 10,random_state = 22,min_value=0)),
            ('rob_scaler',RobustScaler())])

X_Prepared = prep_pipeline.fit_transform(strat_train_set.loc[:,informative_features].copy())

In [ ]:
from sklearn.model_selection import GridSearchCV
elastic_grid = {'alpha':[0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
               'l1_ratio':np.arange(0.0, 1.0, 0.1),
               'tol':np.arange(0.01, .10, 0.01)}
eNet = ElasticNet(random_state=22)
elastic_grid = GridSearchCV(eNet,elastic_grid,cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True,n_jobs=-1)



elastic_grid.fit(X_Prepared,strat_train_set['med_rental_rate'].copy())

In [ ]:
elastic_grid.best_params_

In [ ]:
elastic_grid.best_estimator_

In [ ]:
np.sqrt(-(elastic_grid.best_score_))

Our cross validation mean for the MSE score is actually higher lets see why that might be with a learning curve.Also note that the complete training data is scaled before cross-validation rather than scaled for each fold during cross validation which might affect our above grid search results

In [ ]:
from sklearn.model_selection import learning_curve

train_sizes,train_scores,test_scores = learning_curve(
Ridge(alpha =1 , solver = 'cholesky'), X_Prepared,strat_train_set['med_rental_rate'],
    train_sizes=[50,100,150,200,288],cv=5,scoring='neg_mean_squared_error',random_state=22
)
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

In [ ]:
plt.figure()
plt.title('Learning Curves')
plt.xlabel("Training examples")
plt.ylabel("Score(MSE)")
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)
plt.grid()

plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

plt.legend(loc="best")


Our MSE might converge to a certain limit even as we gather more training data 
this could mean that in order to reduce our MSE we are likely going to need to 
gather further variables of importance in the future.

**Lets check if ensemble methods or boosting might give some improvement**

In [13]:
from sklearn.ensemble import VotingRegressor
elastic_reg = ElasticNet(alpha = 0.1, tol = .01,l1_ratio = 0.5,random_state= 22)
forest_reg = RandomForestRegressor(n_estimators=100,random_state= 22)
voting_reg = VotingRegressor(
            estimators = [('en',elastic_reg),('rf',forest_reg)],
            n_jobs = -1)
#Random Forest Regressor

print('A voting Regressor Ensemble(ElasticNet with Random Forest) 5 fold Cross-Validation Results: ')

model_pipeline(voting_reg,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)

A voting Regressor Ensemble(ElasticNet with Random Forest) 5 fold Cross-Validation Results: 
Scores [[216.19043793 181.9209085  203.31474648 214.55382382 227.5918772 ]]
Mean 208.71435878501106
Standard Deviation 15.449249845405017


An ensemble of our Elastic Net and our Random Forest model increased our RMSE slightly from ~207 to ~208 but reduces our std from ~17 to ~15

How about a Gradient Boosting Regressor

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(
                            X_Prepared,strat_train_set['med_rental_rate'],
                            test_size = 0.2,random_state = 22)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error


gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120,random_state=22)
gbrt.fit(X_train, y_train)

errors = [mean_squared_error(y_val, y_pred)
         for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors)

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators,random_state=22)

In [ ]:
print('A Gradient Boosting Regressor 5 fold Cross-Validation Results: ')
model_pipeline(gbrt_best,strat_train_set,5,feature_selection_done= True,myfeatures = informative_features)

The std for RMSE for a GradientBoostingRegressor is our lowest yet ~11 but it does have a higher mean RMSE from our lowest of 223 

# The System that we will evaluate on the Test Set will be the ensemble model

In [16]:
#making alist of the numerical attributes
num_attrs = informative_features.copy()
del num_attrs[22:24]

In [20]:
%%capture
import joblib
# Save the model as a pickle in a file 
#joblib.dump(winning_pipeline_results['myfinalmodel'], 'trained_final_model.pkl') 
#joblib.dump(winning_pipeline_results['final_predictions'], 'trained_final_model_predictions.pkl')
joblib.dump(strat_train_set['med_rental_rate'], 'training_data_labels.pkl') 
joblib.dump(strat_test_set['med_rental_rate'].copy(), 'test_data_labels.pkl')


Our final Root Mean Squared Error is:

In [17]:
%%capture
winning_pipeline_results = winning_pipeline(strat_train_set,strat_test_set,voting_reg,feature_selection_done=True,myfeatures=informative_features,numerical_attributes=num_attrs)

In [ ]:
winning_pipeline_results['final_rmse']

Our explained variance in median rental rate by our final model is:

In [ ]:
winning_pipeline_results['final_expvar']

How precise is this estimate lets compute a 95% confidence interval for the generalization error

In [ ]:
from scipy import stats
confidence =.95
squared_errors = (winning_pipeline_results['final_predictions']-strat_test_set['med_rental_rate'])**2
np.sqrt(stats.t.interval(confidence,len(squared_errors) - 1,
                        loc = squared_errors.mean(),
                        scale = stats.sem(squared_errors)))


For future research perhaps some categorical variables should be considered after seeking domain expertise about valuable categorical variables to consider in the prediction of median rental price. 

This model may not have produced the desired solution to our business problem however in this project we learned quite a lot. In the story to follow we will summarize the
key findings of this project. 

<a id="story"></a>

# A story about applicable results from the project.

Bob Ross is a 40 year old college professor. He has 1 kid in college and is widowed. Unfortunately for Bob he has had  a series of unexpected financial expenses that have rendered him to temporarily fall behind on his mortgage payments on his 3 bedroom house in [San Luis Obispo](https://www.google.com/search?q=san+luis+obispo&oq=sa&aqs=chrome.0.69i59j69i57j69i59l2j69i60l3j69i65.906j0j7&sourceid=chrome&ie=UTF-8), CA. While talking to his friend Tim about his troubles Tim tells him that he can flip the script by renting his home for a period of time. He explains that this may be a good option when two factors are present: Your home would rent for at or more than your mortgage payment and you were able to find an affordable place to stay. San Luis Obispo is a College Town and Bob believes he can rent his home during the school year while he rents a small apartment to help cover the costs before moving back in. Tim says that to determine the rental price of your property, consult directly with a real estate agent or property management company to take a look at comparable rentals in your area. But before doing so Tim invites Bob over to do some research of their own.Tim is savvy with python and has done some research that he presents to Bob.

He explains that across California the the top 3 predictors of median rental rates are monthly cost of housing for property owners including mortgage payment, taxes, insurance,and utilities, median housing cost for homeowners with a mortgage(including the cost of the mortgage or other debt), and the median real estate taxes paid by owners of homes in the area.

<img src="images/Feature_importance_image.png">

Additionally Tim also shows Bob the figure below and suggest that in San Luis Obispo the  median real estate taxes, median home evaluation, and median housing cost for those who own a mortgage contribute to increasing median rental rate. On the other hand it suggest that in San Luis Obispo median owner cost and average number of rooms for housing units in the area and the fact the percent of all occupied housing units that are owned housing units is 38(%)  contribute to decreasing the median rental rate.

In [ ]:
san_luis_obispo = city_df.loc[city_df.city=='SanLuisObispo'].index[0]
shap.initjs()
shap.force_plot(explainer.expected_value,shap_values[san_luis_obispo,:],housing_shap.iloc[san_luis_obispo,:])

Tim tells Bob that he can come up with a prediction 
of the rental price of his property within a margin of error of 196 dollars that takes into account several pieces of information about his property and the local real estate market.  Bob thanks Tim for his help he tells him that this will give him a better understanding of whether the quote he receives from the consultant is an accurate estimate. Bob then goes to Linda a real estate agent and rather than be in a state of unknowing he knows that the estimate Linda provides to him aligns with the prediction Tim had provided him. As a result Bob has piece of mind that he has received a fair estimate of the rental price of his property.

# Acknowledgements
[Hands-On Machine Learning with Scikit-Learn & TensorFlow](http://shop.oreilly.com/product/0636920142874.do)
https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/#three
    https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf
https://elitedatascience.com/primer
https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec
"Towncharts.com - United States Demographics Data." United States Demographics data. N.p., 15 Dec. 2016. Web. 04 Sep. 2019. <http://www.towncharts.com/>.